In [2]:
import numpy as np

The game is as follows:

--You have a deck of four cards; two are red and two are black
--You start out with $100
--Each turn, a card is removed from the deck (there is no replacement so there are four turns)
--At the start of each turn you are given the chance to bet as much of your money as you like on the color of the card; the returns on all bets are 1:1
--What is the optimal strategy for this game? If there are multiple strategies with this same expected value, which has the least variance?

In [3]:
# define all the possible ways the deck can be shuffled (all have equal probability, as 4
# choose 2 equals 6)

one=['B','B','R','R']
two=['B','R','B','R']
three=['B','R','R','B']
four=['R','B','B','R']
five=['R','B','R','B']
six=['R','R','B','B']

decks = [one,two,three,four,five,six]
decks

[['B', 'B', 'R', 'R'],
 ['B', 'R', 'B', 'R'],
 ['B', 'R', 'R', 'B'],
 ['R', 'B', 'B', 'R'],
 ['R', 'B', 'R', 'B'],
 ['R', 'R', 'B', 'B']]

In [4]:
print np.random.randint(0,6)

0


There is a very clear way to define a strategy for this game. The first round, we have an even 50% chance of guessing correctly because there are two red and two black cards. To minimize variance, let's keep our first round wager to $0 since any amount doesn't increase our expected value. For the second round, it seems like we would want to bet, since we have a 2/3 chance of winning. If the first card is black, we guess red since two of the three remaining cards are red and vice versa if the first card is red. So, it seems like it would be advantageous to bet some amount of money during that round. How much is unclear (that's what we'll find out). The rest of the game is pretty straightforward. If two cards of the same color are drawn on the first two turns, we bet all of our money on each of the last two turns, getting 100 percent returns since we know the colors of the last two cards. If we draw one card of each color in the first two turns, we refrain from making a wager on the third turn, since we have another 50-50 coin flip and want to reduce variance, and bet everything on the final coin flip. Sound complicated? Let's try it out.

In [91]:
# this function's only parameter is the amount we bet in the second round, since we want
# to figure out how much is optimal (would it be best to just go ahead and bet everything
# on the second round since we have 2/3 chance?)

def optimal_game(amount):
    # our optimal strategy takes in how much money to bet on the second
    init = 100
    # we start with one of these six possible card arrangements
    deck = decks[np.random.randint(0,6)]
    
    # now we bet on the second card since we have a 2/3 chance of winning
    if deck[0] != deck[1]:
        init += amount
    else:
        init -= amount
    
    # now we bet on the third round if the first two cards were identical
    if deck[0] == deck[1]:
        init = 2*init
    
    # now we always bet all our money on the fourth round and get everything back
    init = 2*init
    return init
        
        

In [95]:
optimal_game(100)

400

In [132]:
# here's a function to run n simulations of the game with any tested amount

def n_sims(func,amount,n):
    results = []
    for i in range(0,n):
        results.append(func(amount))
    return np.mean(results)

In [133]:
# lets try our game with n=100000 with 2nd round wagers with various increments

In [134]:
n_sims(optimal_game,33.3333,100000)

266.66666690599999

In [122]:
n_sims(optimal_game,10,100000)

266.63679999999999

In [116]:
n_sims(optimal_game,20,100000)

266.46559999999999

In [117]:
n_sims(optimal_game,30,100000)

266.66680000000002

In [118]:
n_sims(optimal_game,50,100000)

266.66199999999998

In [119]:
n_sims(optimal_game,70,100000)

266.42099999999999

In [120]:
n_sims(optimal_game,90,100000)

266.77659999999997

In [115]:
n_sims(optimal_game,100,100000)

266.74400000000003

It seems like all strategies, regardless of what we bet in the second round, approach an expected value of 266.67.

To more clearly illustrate why the two are equivalent, we can condense the original function as follows:

In [37]:
def smaller_optimal_game(amount):
    # our optimal strategy takes in how much money to bet on the second
    init = 100
    # we start with one of these six possible card arrangements
    deck = decks[np.random.randint(0,6)]
    
    # now we bet on the second card since we have a 2/3 chance of winning
    if deck[0] != deck[1]:
        init += amount
    else:
        init -= amount
        # if we lose the second round bet, we consequently can double our remaining value
        # in the third round
        init *= 2
    init *= 2
    return init

In [90]:
smaller_optimal_game(100)

400

Mathematically, we conclude:

E(X) = (2/3)(100 + amount)(2) + (1/3)(100 - amount)(4)
E(X) = 133.33 + 133.33 + (4/3)(amount) - (4/3)(amount)
E(X) = 266.67

The value of "amount" actually doesn't matter at all.

How to minimize variance? Well, it turns out you just have to poke around with numbers until you find one that works. It turns out that betting 33.333 each time will yield a variance of 0. If you win the bet, your money goes to 133.33 and you don't bet on the third round. If you lose the bet, your money falls to 66.67, but you double your money back to 133.33 after winning the third round. Either way, this puts you in the same position for the fourth round, ultimately netting 266.67 each time. Congrats